# TensorBoard

TensorBoard provides the visualization and tooling needed for machine learning experimentation:
    
   * Tracking and visualizing metrics such as loss and accuracy
   * Visualizing the model graph (ops and layers)
   * Viewing histograms of weights, biases, or other tensors as they change over time
   * Projecting embeddings to a lower dimensional space
   * Displaying images, text, and audio data
   * Profiling TensorFlow programs

![](Downloads/tensorboard-word-emb-gif.gif)

![](Downloads/tensorboard-dif-visu-gif.gif)

![](Downloads/tensorboard-img-gif.gif)

## Visualizations
### Scalar Dashboard
    TensorBoard's Scalar Dashboard visualizes scalar statistics that vary over time; for example, you might want to track the model's loss or learning rate. you can compare multiple runs, and the data is organized by tag. The line charts have the following interactions:

    Clicking on the small blue icon in the lower-left corner of each chart will expand the chart

    Dragging a rectangular region on the chart will zoom in

    Double clicking on the chart will zoom out

    Mousing over the chart will produce crosshairs, with data values recorded in the run-selector on the left.

    Additionally, you can create new folders to organize tags by writing regular expressions in the box in the top-left of the dashboard.

### Histogram Dashboard
    The HistogramDashboard displays how the statistical distribution of a Tensor has varied over time. It visualizes data recorded via tf.summary.histogram. Each chart shows temporal "slices" of data, where each slice is a histogram of the tensor at a given step. It's organized with the oldest timestep in the back, and the most recent timestep in front. By changing the Histogram Mode from "offset" to "overlay", the perspective will rotate so that every histogram slice is rendered as a line and overlaid with one another.

### Distribution Dashboard
    The Distribution Dashboard is another way of visualizing histogram data from tf.summary.histogram. It shows some high-level statistics on a distribution. Each line on the chart represents a percentile in the distribution over the data: for example, the bottom line shows how the minimum value has changed over time, and the line in the middle shows how the median has changed. Reading from top to bottom, the lines have the following meaning: [maximum, 93%, 84%, 69%, 50%, 31%, 16%, 7%, minimum]

    These percentiles can also be viewed as standard deviation boundaries on a normal distribution: [maximum, μ+1.5σ, μ+σ, μ+0.5σ, μ, μ-0.5σ, μ-σ, μ-1.5σ, minimum] so that the colored regions, read from inside to outside, have widths [σ, 2σ, 3σ] respectively.

### Image Dashboard
    The Image Dashboard can display pngs that were saved via a tf.summary.image. The dashboard is set up so that each row corresponds to a different tag, and each column corresponds to a run. Since the image dashboard supports arbitrary pngs, you can use this to embed custom visualizations (e.g. matplotlib scatterplots) into TensorBoard. This dashboard always shows you the latest image for each tag.

### Audio Dashboard
    The Audio Dashboard can embed playable audio widgets for audio saved via a tf.summary.audio. The dashboard is set up so that each row corresponds to a different tag, and each column corresponds to a run. This dashboard always embeds the latest audio for each tag.

### Graph Explorer
    The Graph Explorer can visualize a TensorBoard graph, enabling inspection of the TensorFlow model. To get best use of the graph visualizer, you should use name scopes to hierarchically group the ops in your graph - otherwise, the graph may be difficult to decipher. For more information, including examples, see the graph visualizer tutorial.

### Embedding Projector
    The Embedding Projector allows you to visualize high-dimensional data; for example, you may view your input data after it has been embedded in a high- dimensional space by your model. The embedding projector reads data from your model checkpoint file, and may be configured with additional metadata, like a vocabulary file or sprite images. 

### Text Dashboard
    The Text Dashboard displays text snippets saved via tf.summary.text. Markdown features including hyperlinks, lists, and tables are all supported.

## Installation Steps 

    pip install tensorboard
    (upgrade the torch,tensorboard,tensorflow)
    pip install --upgrade torchvision
    pip install --upgrade torch
    pip install --upgrade tensorboard
    pip install --upgrade tensorflow  

**Note**
     
    supported versions 
    pytorch version - greater than 1.1.0
    torchvision version  - greater than 0.3.0
    tensorboard version - greater than 1.15

In [1]:
import torch
import torchvision
from torch.utils.tensorboard import SummaryWriter
from torchvision import datasets, transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
writer = SummaryWriter()

## Example model

Take any model and write model's output to tensorboard writer object . Writer will write the output to ./runs/ directory by default.

In [3]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
trainset = datasets.MNIST('mnist_train', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

In [ ]:
class MNISTClass(nn.Module):
    def __init__(self):
        super(MNISTClass, self).__init__()
        self.conv1 = nn.Conv2d(1, 15, kernel_size=3, stride=1)
        self.conv2 = nn.Conv2d(15, 30, kernel_size=3, stride=2)
        self.fc1 = nn.Linear(1080, 100)
        self.fc2 = nn.Linear(100, 10)

    def forward(self, x):
        # conv1(kernel=3, filters=15) 28x28x1 -> 26x26x15
        x = F.relu(self.conv1(x))
        
        # conv2(kernel=3, filters=20) 26x26x15 -> 13x13x30
        # max_pool(kernel=2) 13x13x30 -> 6x6x30
        x = F.relu(F.max_pool2d(self.conv2(x), 2, stride=2))

        # flatten 6x6x30 = 1080
        x = x.view(-1, 1080)

        # 1080 -> 100
        x = F.relu(self.fc1(x))

        # 100 -> 10
        x = self.fc2(x)

        # transform to logits
        return F.log_softmax(x, dim=1)

In [ ]:
model = MNISTClass()

In [6]:
images, labels = next(iter(trainloader))
grid = torchvision.utils.make_grid(images)

## Image Dashboard

In [ ]:
writer.add_image('images', grid, 0)

## Graph Explorer

In [7]:
writer.add_graph(model, images)

In [8]:
model

MNISTClass(
  (conv1): Conv2d(1, 15, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(15, 30, kernel_size=(3, 3), stride=(2, 2))
  (fc1): Linear(in_features=1080, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=10, bias=True)
)

In [9]:
optimizer = optim.Adam(model.parameters(), lr=0.01)
loss_fn = nn.CrossEntropyLoss()

def get_num_correct(pred,Y):
    return pred.argmax(dim=1).eq(Y).sum().item()

## Scalar ,Histogram and Distribution Dashboard

In [10]:
for epoch in range(10):
    
    total_loss = 0
    total_correct = 0
    
    for i,batch in enumerate(trainloader): 
        
        X,Y = batch
        pred = model(X)
        loss = loss_fn(pred,Y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        total_correct += get_num_correct(pred,Y)
        
        if i>=20:
            break
            
    writer.add_scalar('Loss', total_loss/(i+1), epoch)
    writer.add_scalar('Number Correct', total_correct, epoch)
    writer.add_scalar('Accuracy', total_correct /1344 , epoch)
    
    writer.add_histogram('conv1.bias', model.conv1.bias, epoch)
    writer.add_histogram('conv1.weight', model.conv1.weight, epoch)
    writer.add_histogram(
        'conv1.weight.grad'
        ,model.conv1.weight.grad
        ,epoch
    )
    
    print(
        "epoch", epoch, 
        "total_correct:", total_correct, 
        "loss:", total_loss/(i+1)
    )
    

epoch 0 total_correct: 705 loss: 1.4361859418096996
epoch 1 total_correct: 1193 loss: 0.3890482286612193
epoch 2 total_correct: 1230 loss: 0.26689529028676806
epoch 3 total_correct: 1271 loss: 0.17319698809158235
epoch 4 total_correct: 1265 loss: 0.18897668538349016
epoch 5 total_correct: 1281 loss: 0.14157835873109953
epoch 6 total_correct: 1294 loss: 0.12438674350934369
epoch 7 total_correct: 1288 loss: 0.1539668036358697
epoch 8 total_correct: 1286 loss: 0.13928663588705517
epoch 9 total_correct: 1298 loss: 0.1250829054486184


## Embedding Projector

**Try one of the following**

### 1.Word embedding

In [ ]:
import keyword
import torch
meta = []
while len(meta)<100:
    meta = meta+keyword.kwlist # get some strings
meta = meta[:100]

for i, v in enumerate(meta):
    meta[i] = v+str(i)

label_img = torch.rand(100, 3, 10, 32)
for i in range(100):
    label_img[i]*=i/100.0

writer.add_embedding(torch.randn(100, 5), metadata=meta, label_img=label_img)

### 2.Image embedding

In [ ]:
dataset = datasets.MNIST('mnist', train=False, download=True)
images = dataset.test_data[:100].float()

label = dataset.test_labels[:100]
features = images.view(100, 784)

writer.add_embedding(features, metadata=label, label_img=images.unsqueeze(1))

In [ ]:
writer.close()

## Command for Run

Run the  **tensorboard --logdir=runs** in command prompt.

Open http://localhost:6006/ to see the results.

To quit , press CRTL+C.

**Alternative Method**

    Open from Jupyter-Notebook using magic comments
    %load_ext tensorboard      
    %tensorboard --logdir logs 

In [3]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir logs